## Project Template (Deepnote)

You can use this notebook as a starting point for your class project and/or the extra exercise for graduate students in Deepnote. It comes set up with the dependencies we use in class, so you don't have to install these yourself.

**Notes:**
- To use this in your own workspace, click “Duplicate” for the project in Deepnote.
- Class dependencies, namely `drake` and the `manipulation` package, are already preinstalled in the Deepnote image for this course, as well as a bunch of other dependencies that these packages depend on or that we use in class
- To see the full list of installed dependencies, as well as version numbers etc, see this file: [pyproject.toml](https://github.com/RussTedrake/manipulation/blob/master/pyproject.toml).
  - **Note**: we install all the extra dependencies, including the `dev` dependencies, into the Docker image that this deepnote project runs on.


In [ ]:
%load_ext autoreload
%autoreload 2
# ^ make it so we can change inner files


In [2]:

import numpy as np
from pydrake.all import (
    ContactModel,
    DiagramBuilder,
    MeshcatVisualizer,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    SceneGraph,
    Simulator,
    Sphere,
    StartMeshcat,
    VectorLogSink,
    VisualizationConfig,
)

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    Parser,
)
from scripts.load_scenario import load_scenario

In [3]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click the link above to open Meshcat in your browser!


In [4]:
use_hydroelastic = False # this takes forever

def make_builder():
    scenario = load_scenario()

    # since makehardwarestation finalizes the plant before giving it back to us
    def callback(parser: Parser):
        plant = parser.plant()
        plant.set_contact_model(ContactModel.kHydroelasticWithFallback)
        scene_graph: SceneGraph = parser.scene_graph()
        config = scene_graph.get_config()
        if use_hydroelastic:
            config.default_proximity_properties.compliance_type = "compliant"
        scene_graph.set_config(config)

    station = MakeHardwareStation(scenario, meshcat=meshcat, parser_prefinalize_callback=callback)
    plant: MultibodyPlant = station.plant()


    builder = DiagramBuilder()
    builder.AddSystem(station)

    to_point_cloud = AddPointClouds(
        scenario=scenario, station=station, builder=builder, meshcat=meshcat
    )
    builder.ExportOutput(
        to_point_cloud["camera0"].point_cloud_output_port(), "point_cloud0"
    )

    return builder, plant, station

# diagram = builder.Build()


In [6]:
from scripts.grasp_broom import plan_path, build_temp_plant, combine_trajectory

# basic test
# X_WG1 = RigidTransform(RollPitchYaw(np.pi/2, 0, 0), [0, 0.5, 1])
# X_WG2 = RigidTransform(RollPitchYaw(np.pi/2, 0, np.pi/2), [0, 1, 1])

# meshcat.SetObject("start", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("start", X_WG1)
# meshcat.SetObject("goal", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("goal", X_WG2)


In [5]:
# test function to see the temp plant

# diagram, plant, gripper_frame = build_temp_plant(q0 = None, meshcat = meshcat)
# config = VisualizationConfig(lcm_bus='default', publish_period=0.015625, publish_illustration=True, default_illustration_color=Rgba(r=0.9, g=0.9, b=0.9, a=1.0), publish_proximity=True, default_proximity_color=Rgba(r=0.8, g=0.0, b=0.0, a=1.0), initial_proximity_alpha=0.5, publish_contacts=True, publish_inertia=True, enable_meshcat_creation=True, delete_on_initialization_event=True, enable_alpha_sliders=False)
# context = diagram.CreateDefaultContext()
# diagram.ForcedPublish(context)
# meshcat.SetObject("gripper", Sphere(0.02), rgba=Rgba(0.1, 0.9, 0.1, 1))
# meshcat.SetTransform("gripper", gripper_frame.GetPoseInWorld(context))


In [ ]:
# path = plan_path(X_WG1, X_WG2)

In [ ]:
from pydrake.systems.primitives import TrajectorySource, ConstantVectorSource
from scripts.ik import solve_ik_for_pose

builder, plant, station = make_builder()

# grasp broom
X_WStart = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0, 1.0, 0.5])
X_WGoal = RigidTransform(RollPitchYaw(np.pi, 0, 0), [0.6, 1.6, 0.5])

AddMeshcatTriad(meshcat, 'a', X_PT=X_WStart)
AddMeshcatTriad(meshcat, 'b', X_PT=X_WMid)
AddMeshcatTriad(meshcat, 'c', X_PT=X_WGoal)

traj = plan_path(X_WStart, X_WGoal)

# print 20 joint positions sampled
times = np.linspace(traj.start_time(), traj.end_time(), 20)
for t in times:
    print(f'time{t}: {traj.value(t).flatten()}')

# add traj to builder and connect
q_src = builder.AddSystem(TrajectorySource(traj))
builder.Connect(q_src.get_output_port(), station.GetInputPort("iiwa.position"))
wsg_src = builder.AddSystem(ConstantVectorSource([1.0]))
builder.Connect(wsg_src.get_output_port(), station.GetInputPort("wsg.position"))

diagram = builder.Build()


7
(7, 10)
Collision optimization failed
time0.0: [ 0.65299455 -0.54730929 -1.51971125 -1.63925123 -1.11975078  1.53316186
  0.00306206]
time0.12477860219717837: [ 0.71104527 -0.49940633 -1.54257428 -1.62431343 -1.1333649   1.54592975
  0.00269774]
time0.24955720439435675: [ 0.85101524 -0.38584658 -1.57877298 -1.58112517 -1.15018425  1.57211636
  0.00198246]
time0.3743358065915351: [ 1.02178636e+00 -2.51706025e-01 -1.58008123e+00 -1.51219621e+00
 -1.13442084e+00  1.59364437e+00  1.48488073e-03]
time0.4991144087887135: [ 1.1857594  -0.12919498 -1.52959644 -1.42631465 -1.07176487  1.60099704
  0.00195854]
time0.6238930109858919: [ 1.32915322 -0.02785645 -1.46560288 -1.34332996 -0.98574754  1.59973996
  0.00448273]
time0.7486716131830702: [ 1.44089169  0.04592459 -1.42960569 -1.28321552 -0.90278155  1.59674477
  0.01008653]
time0.8734502153802486: [ 1.5146111   0.10005227 -1.42285272 -1.24911289 -0.83297952  1.59503287
  0.01839852]
time0.998228817577427: [ 1.54785796  0.15428741 -1.413187

In [18]:

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.Initialize()
simulator.AdvanceTo(traj.end_time())
meshcat.StopRecording()
meshcat.PublishRecording()